<a href="https://colab.research.google.com/github/myazdani/formal-algorithms-for-transformers/blob/main/modulo_math.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed May  3 05:49:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/myazdani/formal-algorithms-for-transformers.git

fatal: destination path 'formal-algorithms-for-transformers' already exists and is not an empty directory.


In [3]:
#!pip install -r ./formal-algorithms-for-transformers/requirements.txt
!wget -O reqs.txt https://raw.githubusercontent.com/myazdani/regularized-t-learner/main/requirements.txt

--2023-05-03 05:50:01--  https://raw.githubusercontent.com/myazdani/regularized-t-learner/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 107 [text/plain]
Saving to: ‘reqs.txt’

reqs.txt            100%[===================>]     107  --.-KB/s    in 0s      

2023-05-03 05:50:01 (7.32 MB/s) - ‘reqs.txt’ saved [107/107]



In [4]:
%%capture
!pip install -r reqs.txt

In [5]:
!pip3 install numpy --pre torch[dynamo] --force-reinstall --extra-index-url https://download.pytorch.org/whl/nightly/cu118

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/nightly/cu118
  Using cached numpy-1.24.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached https://download.pytorch.org/whl/nightly/cu118/torch-2.1.0.dev20230501%2Bcu118-cp310-cp310-linux_x86_64.whl (2268.2 MB)
  Using cached https://download.pytorch.org/whl/nightly/pytorch_triton-2.1.0%2B7d1a95b046-cp310-cp310-linux_x86_64.whl (88.8 MB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Using cached filelock-3.12.0-py3-none-any.whl (10 kB)
  Using cached https://download.pytorch.org/whl/nightly/fsspec-2023.4.0-py3-none-any.whl (153 kB)
  Using cached https://download.pytorch.org/whl/nightly/Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached sympy-1.12rc1-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached https://download.pytorch.org/whl/nightly/MarkupSafe-2.1

In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import sys
sys.path.insert(0, './formal-algorithms-for-transformers')
from src.alg_5 import MHAttentionInefficient
from src.alg_10 import DTransformer

In [7]:
torch.__version__

'2.1.0.dev20230501+cu118'

In [8]:
import pytorch_lightning as pl


In [9]:
from typing import Any

In [10]:
class GPT(pl.LightningModule):

    def __init__(self, embed_dim, mlp_dim, max_seq_len, L_dec, vocab_size, num_heads,
                 grad_glip: float = 0.9, optimizer: str = "AdamW", learning_rate: float = 1e-3, 
                 lr_scheduler: str = None,**kwargs: Any
                 ):
        super().__init__()
        self.save_hyperparameters()        
        self.model = DTransformer(embed_dim=embed_dim, 
                                  mlp_dim=mlp_dim, 
                                  max_seq_len=max_seq_len,
                                  L_dec=L_dec, 
                                  vocab_size=vocab_size, 
                                  num_heads=num_heads
                                  )

        self.criterion = nn.NLLLoss()
    

    def forward(self, x):
        output = self.model(x)
        return output

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(torch.log(y_hat[:,-1,:]), y)
        return loss

    def configure_optimizers(self):
        # ref: https://github.com/Lightning-AI/lightning/issues/7576
        optimizer = getattr(torch.optim, self.hparams.optimizer)(
            self.parameters(),
            lr=self.hparams.learning_rate,
        )
        if self.hparams.lr_scheduler is None:
            return optimizer
        scheduler = self.configure_scheduler(optimizer, self.hparams.lr_scheduler)
        return [optimizer], [scheduler]        




In [11]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class Digits(Dataset):
    def __init__(self, vocab_size, N, max_seq_len):
        super().__init__()
        self.x_ids = torch.randint(0,vocab_size, size = (N, max_seq_len)) 
        self.y_ids = self.x_ids.sum(1)%vocab_size
    def __len__(self):
        return len(self.x_ids)

    def __getitem__(self, idx):
        return self.x_ids[idx],self.y_ids[idx]

In [12]:

tr_loader = DataLoader(Digits(max_seq_len=2, vocab_size=12, N=32), 
                       batch_size=10000, shuffle=False)    
vl_loader = DataLoader(Digits(max_seq_len=2, vocab_size=12, N=1024), 
                       batch_size=10000, shuffle=True)    

In [13]:
trainer = pl.Trainer(max_epochs=1000)
modulo_adder = GPT(embed_dim=50, mlp_dim=64, max_seq_len=2, L_dec=5, vocab_size=12, 
                   num_heads=5)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [14]:
%%time
trainer.fit(model=modulo_adder, train_dataloaders=tr_loader, 
            #val_dataloaders=vl_loader
            )

INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type         | Params
-------------------------------------------
0 | model     | DTransformer | 288 K 
1 | criterion | NLLLoss      | 0     
-------------------------------------------
288 K     Trainable params
0         Non-trainable params
288 K     Total params
1.156     Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloa

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1000` reached.


CPU times: user 1min 43s, sys: 6.32 s, total: 1min 49s
Wall time: 1min 46s


In [15]:
for batch in tr_loader:
    break
x, y = batch   
y_hat = modulo_adder(x) 
torch.mean((y_hat[:,-1,:].argmax(1) == y).float())

tensor(0.9375)

testing to see if torch.compile gives any speedup

In [17]:
trainer = pl.Trainer(max_epochs=1000)
modulo_adder = GPT(embed_dim=50, mlp_dim=64, max_seq_len=2, L_dec=5, vocab_size=12, 
                   num_heads=5)
modulo_adder = torch.compile(modulo_adder)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [18]:
%%time
trainer.fit(model=modulo_adder, train_dataloaders=tr_loader, 
            #val_dataloaders=vl_loader
            )

INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type         | Params
-------------------------------------------
0 | model     | DTransformer | 288 K 
1 | criterion | NLLLoss      | 0     
-------------------------------------------
288 K     Trainable params
0         Non-trainable params
288 K     Total params
1.156     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_inductor/compile_fx.py:108: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1000` reached.


CPU times: user 1min 59s, sys: 10.1 s, total: 2min 9s
Wall time: 2min 9s


In [19]:
for batch in tr_loader:
    break
x, y = batch   
y_hat = modulo_adder(x) 
torch.mean((y_hat[:,-1,:].argmax(1) == y).float())

tensor(0.9375)